# DSC530-T302
# Weeks 5-6 Assignment
# Jacob Siau

## Chapter 4 Exercises 1-2, 4, 6-7, 10

### Exercise 1.

With the earthquakes.csv file, select all the earthquakes in Japan with a magnitude of 4.9 or greater using the mb magnitude type.

In [ ]:
import pandas as pd

# Load the earthquakes data
df = pd.read_csv('earthquakes.csv')

# Select all earthquakes in Japan with a magnitude of 4.9 or greater using the mb magnitude type
japan_earthquakes = df[(df['parsed_place'] == 'Japan') & (df['mag'] >= 4.9) & (df['magType'] == 'mb')]
print(japan_earthquakes)

      mag magType           time                         place  tsunami  \
1563  4.9      mb  1538977532250  293km ESE of Iwo Jima, Japan        0   
2576  5.4      mb  1538697528010    37km E of Tomakomai, Japan        0   
3072  4.9      mb  1538579732490     15km ENE of Hasaki, Japan        0   
3632  4.9      mb  1538450871260    53km ESE of Hitachi, Japan        0   

     parsed_place  
1563        Japan  
2576        Japan  
3072        Japan  
3632        Japan  


### Exercise 2. 
Create bins for each full number of earthquake magnitude (for instance, the first bin is (0,1], the second is (1,2], and so on) with the ml magnitude type and count how many are in each bin.

In [ ]:
# Create bins for each full number of earthquake magnitude with ml magnitude type
ml_earthquakes = df[df['magType'] == 'ml']

# Create bins from 0 to the max magnitude (rounded up)
bins = range(0, int(ml_earthquakes['mag'].max()) + 2)

# Count earthquakes in each bin
magnitude_bins = pd.cut(ml_earthquakes['mag'], bins=bins)
bin_counts = magnitude_bins.value_counts().sort_index()

print("Earthquake counts by magnitude bin (ml type):")
print(bin_counts)

Earthquake counts by magnitude bin (ml type):
mag
(0, 1]    2207
(1, 2]    3105
(2, 3]     862
(3, 4]     122
(4, 5]       2
(5, 6]       1
Name: count, dtype: int64


### Exercise 4.
Build a crosstab with the earthquake data between the tsunami column and the magType column. Rather than showing the frequency count, show the maximum magnitude that was observed for each combination. Put the magnitude type along the columns.

In [ ]:
# Build a crosstab between tsunami and magType columns showing maximum magnitude
crosstab = pd.crosstab(df['tsunami'], df['magType'], values=df['mag'], aggfunc='max')
print("Crosstab of tsunami vs magType (showing max magnitude):")
print(crosstab)

Crosstab of tsunami vs magType (showing max magnitude):
magType   mb  mb_lg    md   mh   ml  ms_20    mw  mwb  mwr  mww
tsunami                                                        
0        5.6    3.5  4.11  1.1  4.2    NaN  3.83  5.8  4.8  6.0
1        6.1    NaN   NaN  NaN  5.1    5.7  4.41  NaN  NaN  7.5


### Exercise 6.
Create a pivot table of the FAANG data that compares the stocks. Put the ticker in the rows and show the averages of the OHLC and volume traded data.

In [ ]:
# Load the FAANG data
faang_df = pd.read_csv('faang.csv')

# Create a pivot table with ticker in rows and averages of OHLC and volume
pivot_table = faang_df.pivot_table(
    index='ticker',
    values=['open', 'high', 'low', 'close', 'volume'],
    aggfunc='mean'
)

print("Pivot table of FAANG data (averages of OHLC and volume):")
print(pivot_table)

Pivot table of FAANG data (averages of OHLC and volume):
              close         high          low         open        volume
ticker                                                                  
AAPL      47.263357    47.748526    46.795877    47.277859  1.360803e+08
AMZN    1641.726176  1662.839839  1619.840519  1644.072709  5.648994e+06
FB       171.510956   173.613347   169.303148   171.472948  2.765860e+07
GOOG    1113.225134  1125.777606  1101.001658  1113.554101  1.741965e+06
NFLX     319.290319   325.219322   313.187330   319.620558  1.146962e+07


### Exercise 10.
The European Centre for Disease Prevention and Control (ECDC) provides an open dataset on COVID-19 cases called daily number of new reported cases of COVID-19 by country worldwide. This dataset is updated daily, but we will use a snapshot that contains data through September 18, 2020. Complete the following tasks to practice the skills you've learned up to this point in the book:

Prepare the data
Read in the data in the covid19_cases.csv file.
Create a date column by parsing the dateRep column into a datetime.
Set the date column as the index.
Use the replace() method to update all occurrences of United_States_of_America and United_Kingdom to USA and UK, respectively.
Sort the index.
For the five countries with the most cases (cumulative), find the day with the largest number of cases.
Find the 7-day average change in COVID-19 cases for the last week in the data for the five countries with the most cases.
Find the first date that each country other than China had cases.
Rank the countries by cumulative cases using percentiles.

In [ ]:
# Load the COVID-19 data
covid_df = pd.read_csv('covid19_cases.csv')

# Create a date column by parsing the dateRep column into a datetime
covid_df['date'] = pd.to_datetime(covid_df['dateRep'], format='%d/%m/%Y')

# Set the date column as the index
covid_df.set_index('date', inplace=True)

# Replace country names
covid_df['countriesAndTerritories'] = covid_df['countriesAndTerritories'].replace({
    'United_States_of_America': 'USA',
    'United_Kingdom': 'UK'
})

# Sort the index
covid_df.sort_index(inplace=True)

# Find the five countries with the most cumulative cases
top_5_countries = covid_df.groupby('countriesAndTerritories')['cases'].sum().nlargest(5).index.tolist()
print("Top 5 countries with most cases:", top_5_countries)

# For the five countries with the most cases, find the day with the largest number of cases
print("\nDay with largest number of cases for top 5 countries:")
for country in top_5_countries:
    country_data = covid_df[covid_df['countriesAndTerritories'] == country]
    max_cases_day = country_data['cases'].idxmax()
    max_cases = country_data.loc[max_cases_day, 'cases']
    print(f"{country}: {max_cases_day.date()} with {max_cases} cases")

# Find the 7-day average change in COVID-19 cases for the last week for top 5 countries
print("\n7-day average change in COVID-19 cases for the last week:")
for country in top_5_countries:
    country_data = covid_df[covid_df['countriesAndTerritories'] == country].sort_index()
    last_week = country_data.tail(7)
    avg_change = last_week['cases'].diff().mean()
    print(f"{country}: {avg_change:.2f}")

# Find the first date that each country other than China had cases
print("\nFirst date each country (other than China) had cases:")
first_cases = covid_df[covid_df['countriesAndTerritories'] != 'China'].groupby('countriesAndTerritories').apply(
    lambda x: x[x['cases'] > 0].index.min()
).dropna().sort_values()
print(first_cases.head(10))

# Rank the countries by cumulative cases using percentiles
cumulative_cases = covid_df.groupby('countriesAndTerritories')['cases'].sum()
percentile_ranks = cumulative_cases.rank(pct=True) * 100
print("\nCountry rankings by cumulative cases (percentiles):")
print(percentile_ranks.sort_values(ascending=False).head(10))

Top 5 countries with most cases: ['USA', 'India', 'Brazil', 'Russia', 'Peru']

Day with largest number of cases for top 5 countries:
USA: 2020-07-25 with 78427 cases
India: 2020-09-17 with 97894 cases
Brazil: 2020-07-30 with 69074 cases
Russia: 2020-07-18 with 12640 cases
Peru: 2020-08-17 with 10143 cases

7-day average change in COVID-19 cases for the last week:
USA: 1564.83
India: -172.50
Brazil: 1045.67
Russia: 69.50
Peru: 25.33

First date each country (other than China) had cases:
countriesAndTerritories
Thailand      2020-01-13
Japan         2020-01-15
South_Korea   2020-01-20
Taiwan        2020-01-21
USA           2020-01-21
Singapore     2020-01-24
Vietnam       2020-01-24
Malaysia      2020-01-25
Nepal         2020-01-25
Australia     2020-01-25
dtype: datetime64[ns]

Country rankings by cumulative cases (percentiles):
countriesAndTerritories
USA             100.000000
India            99.523810
Brazil           99.047619
Russia           98.571429
Peru             98.095238
C

C:\Users\Jacob\AppData\Local\Temp\ipykernel_7384\3061757763.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_cases = covid_df[covid_df['countriesAndTerritories'] != 'China'].groupby('countriesAndTerritories').apply(
